In [20]:
import numpy as np
from numpy import sqrt, array, exp

def get_alpha_i(a, expiry, sigma, hw_dates):
    ''' a - HW mean rev. aka (k)
        T - expiry '''
    assert isinstance(a, float), 'mean reversion should be float'
    assert isinstance(expiry, float), 'expiry should be float'
    assert isinstance(hw_dates, np.ndarray), 'hw_dates should be np.ndarray'
    assert isinstance(sigma, float) or isinstance(sigma, dict), 'sigma should be float or dict'

    if isinstance(sigma, float):
        variance_term = np.sqrt(0.5 * (sigma ** 2) * (np.exp(2 * a * expiry) - 1) / a)
        res = [variance_term * (np.exp(-a*expiry) - np.exp(-a*t))/a for t in hw_dates]
        return np.array(res)
    elif isinstance(sigma, dict):
        expiries = sigma['buckets'][1:-1]
        sigmas = sigma['values'][1:-1]
        variance_term = 0.5 * (sigmas[0] ** 2) * (np.exp(2 * a * expiries[0]) - 1) / a
        for i in range(len(expiries) - 1):
            variance_term += 0.5 * (sigmas[i+1] ** 2) * (np.exp(2 * a * expiries[i+1]) - np.exp(2 * a * expiries[i])) / a

        res = [sqrt(variance_term) * (np.exp(-a * expiry) - np.exp(-a * t)) / a for t in hw_dates]
        return np.array(res)

In [30]:
a = 0.05
expiry = 0.99726027
const_sigma = 50.
hw_dates = np.array([1.00547945, 2.0109589 , 3.00821918, 4.00821918, 5.00821918])

ref_sigma = {
    'buckets': np.array([ 0.        , 0.99726027, 30.99726027]),
    'values': np.array([const_sigma,const_sigma, const_sigma])}

In [31]:
ref_sigma2 = {
    'buckets':np.array([ 0. ,     0.3,        0.5, 0.99726027, 30.99726027]),
    'values': np.array([const_sigma, const_sigma, const_sigma, const_sigma, const_sigma])
}

In [32]:
get_alpha_i(a, expiry, ref_sigma, hw_dates)

array([  0.40029105,  48.14884517,  93.19423327, 136.16341475,
       177.03696451])

In [33]:
get_alpha_i(a, expiry, ref_sigma2, hw_dates)

array([  0.40029105,  48.14884517,  93.19423327, 136.16341475,
       177.03696451])